In [4]:
import numpy as np
import pandas as pd
import tables

In [31]:
# data = pd.read_hdf('data.h5')
# sales_train_evaluation.csv

In [8]:
INPUT_DIR = '../../input/m5-forecasting-accuracy/'
OUTPUT_DIR =  '../../input/lv1/'

In [5]:
cal = pd.read_csv(INPUT_DIR+'calendar.csv')

In [6]:
sales = pd.read_csv(INPUT_DIR+'sales_train_evaluation.csv')

In [7]:
# week day feature
cal_ = cal.loc[:,['wday','month','event_name_1','snap_CA','snap_TX','snap_WI']]
cal_['day_type'] = cal_['wday'].map({1: 5,2:6,3: 0,4:2,5:2,6:2,7:4})
cal_.loc[cal_['event_name_1'].isna(), 'holi_bool'] = 0
cal_.loc[~cal_['event_name_1'].isna(), 'holi_bool'] = 1
cal_['holi_-1'] = cal_["holi_bool"].shift(1)
cal_['holi_1'] = cal_["holi_bool"].shift(-1)
cal_.loc[cal_['holi_-1']==1, 'day_type'] = 8
cal_.loc[cal_['holi_1']==1, 'day_type'] = 8
cal_.loc[cal_['holi_bool']==1, 'day_type'] = 7
cal_.drop(['wday','month','event_name_1','holi_bool','holi_-1','holi_1'], axis=1, inplace=True)

In [10]:
cal_.to_csv(OUTPUT_DIR+'cal_feat.csv',index=False)

In [11]:
cal_.T

,0,1,2,3,4,5,6,7,8,9,...,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968
snap_CA,0,0,0,1,1,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
snap_TX,0,0,0,1,0,1,0,1,1,1,...,0,1,1,1,0,1,0,0,0,0
snap_WI,0,0,0,0,1,1,0,1,1,0,...,0,1,1,0,1,1,0,0,0,0
day_type,5,6,0,2,2,2,4,8,7,8,...,4,5,6,0,2,2,2,4,8,7


In [ ]:
price_df = pd.read_csv(INPUT_DIR+'sell_prices.csv')
cal_df = pd.read_csv(INPUT_DIR+'calendar.csv')
sales = pd.read_csv(INPUT_DIR+'sales_train_evaluation.csv')
store_list = price_df['store_id'].unique().tolist()
for store in store_list:
    print(store)
    p_df = pd.merge(price_df, cal_df.loc[:,['wm_yr_wk', 'd']], on='wm_yr_wk')
    p_df = p_df[p_df['store_id']==store].loc[:,['item_id','sell_price','d']]
    p_df = p_df.set_index(['item_id','d'])
    p_df = p_df.unstack(level='d').rename(columns= lambda x: x[2:])
    levels = p_df.columns.levels
    labels = p_df.columns.labels
    p_df.columns = levels[1][labels[1]]
    new_idx=[str(i+1) for i in range(1969)]
    p_df = p_df.loc[:,new_idx]
    p_df = pd.merge(sales.loc[:,'item_id'], p_df, on='item_id', how='left')
    p_df = p_df.fillna(0)
    p_df.to_csv(f"{OUTPUT_DIR}price_{store}.csv",index=False)

In [ ]:
p_df.fillna(0)